In [ ]:
import sys
sys.path.append("/storage/vbutoi/projects/")
sys.path.append("/storage/vbutoi/libraries/")
sys.path.append("/storage/vbutoi/projects/UniverSeg/")
sys.path.append("/storage/vbutoi/projects/DiSeg/")

from IonPy.analysis import ResultsLoader
import seaborn as sns

sns.set_style("darkgrid")


import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage',
       '/storage/megamedical'
))

# Results loader object does everything
rs = ResultsLoader()
root = "/storage/vbutoi/scratch"

Get the random baseline number for our model.

In [ ]:
from DiSeg.diseg.experiment.analysis.plot import load_baseline_and_bounds, insert_baseline_and_bounds

b_and_b = load_baseline_and_bounds(task="WBC_CV_0", use_cached=True)

graph_config = {
    #"x_lim": [0, 2950],
    #"y_lim": [0.5, 0.93],
    "b_and_b": b_and_b,
    "colors":{
        'random': 'cyan',
        'xx': 'red',
        'yy': 'green',
        'upper-bound': 'purple',
    }
}

In [ ]:
import pandas as pd

path = f"{root}/05-30-23_WeDontNeedNoEducation"

dfc = rs.load_configs(
    path,
    properties=False,
)
df = rs.load_metrics(dfc)
smooth_df = df.loc[df['epoch'] % 50 == 0].copy()
smooth_df = smooth_df.where(pd.notna(smooth_df), None)

def query_split(phase):
    if phase == "train":
        return "val"
    else:
        return "test"
    
smooth_df.augment(query_split)

g = sns.relplot(
    data=smooth_df,
    x='epoch',
    y='dice_score',
    col='query_split',
    hue='use_label',
    kind='line',
    height=12,
    )

# Set column spacing
g.fig.subplots_adjust(wspace=0.05)
insert_baseline_and_bounds(graph=g, **graph_config)